In [1]:
import paho.mqtt.client as mqtt
from optparse import OptionParser
from collections import defaultdict
import time
import gc

In [24]:
client_id = '3310-u6142160'

client_name = '3310student'
client_password = 'comp3310'

server_addr = '3310exp.hopto.org'

sys_topic = '$SYS/#'
sys_key   = 'SYS'

speed = 'fast'
channel_list = ['q2']
qos_list = [2]
mis_order_buf = 10

In [47]:
init_topic_record = lambda : {'cnt':0, 'valid_cnt':0, 'loss_cnt':0, 'dup_cnt':0, 'mis_order_cnt':0, 
                              'cur_val':-1, 'newest_T':-1, 
#                               'lost_val':defaultdict(lambda:0), 'hist':[], 'wrapping':False,
                              'start_time':time.time(), 'end_time':-1}

def handle_msg(client, userdata, msg):
    for k in msg.__slots__:
        print(k, getattr(msg, k), end=';')
    print()

# client's event on receiving message
def on_message_qos_2(client, userdata, msg):
    # qos=2 => no loss & out-of-order & dup
    
    record = userdata[msg.topic]
    record['cnt'] += 1
    
    try:
        cur_val = int(msg.payload)
#         record['hist'].append(cur_val)
    except:
        print("received ", msg.payload)
        return

    print('msg:', cur_val, msg.timestamp, 'rec:', record['cur_val'], record['newest_T'])
    if record['cur_val'] != -1:

        assert cur_val == record['cur_val'] + 1
        assert msg.timestamp >= record['newest_T']

    record['valid_cnt'] += 1
    record['newest_T'] = msg.timestamp
    record['cur_val'] = cur_val


    gc.collect()
    
# client's event on receiving message
def on_message_qos_1(client, userdata, msg):
    # qos=1 => no loss & out-of-order
    
    record = userdata[msg.topic]
    record['cnt'] += 1
    
    try:
        cur_val = int(msg.payload)
#         record['hist'].append(cur_val)

        if msg.timestamp > record['newest_T']: # in time series
            record['newest_T'] = msg.timestamp
            
            if cur_val == 0 or record['cur_val'] == -1: # wrap around or 1st msg
                record['cur_val'] = cur_val
                record['valid_cnt'] += 1
                
            else:
                if cur_val > record['cur_val']:
                    assert cur_val == record['cur_val'] + 1
                    record['cur_val'] = cur_val
                    record['valid_cnt'] += 1
                else:
                    record['dup_cnt'] += 1
        else: # should come in the past
            record['dup_cnt'] += 1
            

    except:
        print("received ", msg.payload)
    gc.collect()
    
# client's event on receiving message
def on_message_qos_0(client, userdata, msg):
    # qos=0 => no dup
    
    record = userdata[msg.topic]
    record['cnt'] += 1
    
    try:
        cur_val = int(msg.payload)
#         record['hist'].append(cur_val)
        if msg.timestamp > record['newest_T']: # expected order => increasing number

            if cur_val == record['cur_val']+1 or record['cur_val'] == -1 or cur_val == 0: # expected
                # expected or 1st msg or wrap around
                record['valid_cnt'] += 1
            elif cur_val > record['cur_val']: # lossing in-between
                record['loss_cnt'] += cur_val - record['cur_val'] - 1
            else: # lossing 0...
                record['loss_cnt'] += cur_val-1

            record['cur_val'] = cur_val
            record['newest_T'] = msg.timestamp
            
        else: # out-of-order => not lost
            record['mis_order_cnt'] += 1
            record['loss_cnt'] -= 1
                    
    except:
        print("received ", msg.payload)
    gc.collect()

def publish_result(userdata):
    # t 10-minute intervals under
    # studentreport/<your.Uni.ID>/ with the ‘retain’ flag set and QoS=2.
    
    topic = 'studentreport/u6142160'
    qos   = 2
    rt_f  = True
    sub_topic_list = ['/language', '/timestamp']
    
    time_stamp = str(time.time())
    sys_record = userdata['SYS']
    for key in sys_record:
        asd

def subscribe_counter(client, userdata, speed, channel_list, qos_list):
    topic    = 'counter/' + speed + '/'
    for chn, qos in zip(channel_list,qos_list):
        cur_topic = topic + chn
        
        sub_rst = None
        while(sub_rst != mqtt.MQTT_ERR_SUCCESS):
            sub_rst, _ = client.subscribe(cur_topic, qos=qos)
            
        # init record after successfullt subscribe => deteministic time shift of actual successful subscribe
        userdata[cur_topic] = init_topic_record()
        print('sucessfully subscribe ', cur_topic)
# client's event on connect
def on_connect(client, userdata, flags, rc):
    '''
        flags['session present'] = 1 => last session retained by broker, otherwise a new session

        rc 0=Connection successful, 1=incorrect protocol version, 2=invalid client identifier,
           3=server unavailable,    4=bad username or password,   5=not authorised

        userdata: created along with Client class initiliasation
    '''

    # subscribe topic
    subscribe_counter(client, userdata, speed, channel_list, qos_list)
    
#     # subscribe sys
#     client.subscribe(sys_topic, qos=2)
#     userdata[sys_key] = {}
    
# client's event on disconnect
def on_disconnect(client, userdata, rc):
    end_time = time.time()
    for k in userdata.keys():
        userdata[k]['end_time'] = end_time
    
    # print log
    if rc != 0:
        print('Unexpected disconnection: rc =', rc)
    print('log:')
    for k in userdata.keys():
        print(userdata[k])

on_message = lambda qos: on_message_qos_0 if qos == 0 else on_message_qos_1 if qos == 1 else on_message_qos_2
on_message = on_message(min(qos_list))
print(on_message)

<function on_message_qos_2 at 0x7f1f264ef400>


In [54]:
# initialisation
client = mqtt.Client(client_id, clean_session=True, userdata={}, transport="tcp")
client.username_pw_set(username=client_name, password=client_password)

client.on_connect    = on_connect
# client.on_subscribe  = on_subscribe
client.on_message    = on_message
client.on_disconnect = on_disconnect

# potentially blocking => connect_async for non-blocking
client.connect(host=server_addr, port=1883, keepalive=60, bind_address="")
# keepalive:
#     maximum period in seconds allowed between communications with the broker. 
#     If no other messages are being exchanged, this controls the rate at which the client will send 
#     ping messages to the broker

gc.collect()
client.loop_start()

sucessfully subscribe  counter/fast/q2
msg: 194302 91704.276385189 rec: -1 -1
msg: 194303 91704.27662095 rec: 194302 91704.276385189
msg: 194304 91704.276874018 rec: 194303 91704.27662095
msg: 194305 91704.277000569 rec: 194304 91704.276874018
msg: 194306 91704.277187354 rec: 194305 91704.277000569
msg: 194307 91704.277302826 rec: 194306 91704.277187354
msg: 194308 91704.277411653 rec: 194307 91704.277302826
msg: 194309 91704.277519295 rec: 194308 91704.277411653
msg: 194310 91704.27771986 rec: 194309 91704.277519295
msg: 194311 91704.277833984 rec: 194310 91704.27771986
msg: 194312 91704.277941209 rec: 194311 91704.277833984
msg: 194313 91704.278046845 rec: 194312 91704.277941209
msg: 194314 91704.278152054 rec: 194313 91704.278046845
msg: 194315 91704.278259731 rec: 194314 91704.278152054
msg: 194316 91704.357231495 rec: 194315 91704.278259731
msg: 194317 91704.357405061 rec: 194316 91704.357231495
msg: 194318 91704.357503053 rec: 194317 91704.357405061
msg: 194319 91704.357591134 re

Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/admin-u6142160/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/admin-u6142160/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/admin-u6142160/Projects/Web_Crawler/MQTT-Inpsector/paho/mqtt/client.py", line 2650, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "/home/admin-u6142160/Projects/Web_Crawler/MQTT-Inpsector/paho/mqtt/client.py", line 1481, in loop_forever
    rc = self.loop(timeout, max_packets)
  File "/home/admin-u6142160/Projects/Web_Crawler/MQTT-Inpsector/paho/mqtt/client.py", line 1003, in loop
    rc = self.loop_read(max_packets)
  File "/home/admin-u6142160/Projects/Web_Crawler/MQTT-Inpsector/paho/mqtt/client.py", line 1284, in loop_read
    rc = self._packet_read()
  File "/home/admin-u6142160/Projects/Web_Crawler/MQTT-Inpsector/paho/mqtt/client.py", 

In [53]:
# disconnect
client.loop_stop()
client.disconnect()
gc.collect()

msg: 194084 91702.393929874 rec: 194083 91702.393882508
msg: 194085 91702.393976829 rec: 194084 91702.393929874
msg: 194086 91702.474554201 rec: 194085 91702.393976829
msg: 194087 91702.474741705 rec: 194086 91702.474554201
msg: 194088 91702.4748761 rec: 194087 91702.474741705
msg: 194089 91702.475000033 rec: 194088 91702.4748761
msg: 194090 91702.475121389 rec: 194089 91702.475000033
msg: 194091 91702.475233684 rec: 194090 91702.475121389
msg: 194092 91702.47532243 rec: 194091 91702.475233684
msg: 194093 91702.475409287 rec: 194092 91702.47532243
log:
{'cnt': 20, 'valid_cnt': 20, 'loss_cnt': 0, 'dup_cnt': 0, 'mis_order_cnt': 0, 'cur_val': 194093, 'newest_T': 91702.475409287, 'start_time': 1526124874.9782119, 'end_time': 1526124875.879227}


0

In [ ]:
client.publish(sys_topic.rstrip('#'), payload=client_id, qos=0, retain=False)

In [ ]:
269955-273792